<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/VGG_4classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/eyes.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_d')  # Replace with desired destination directory

In [4]:
base_dir = '/content/unzipp_d/eyes'

train_dir = '/content/unzipp_d/eyes/train'
cat_train_dir = '/content/unzipp_d/eyes/train/cataract'
healthy_train_dir = '/content/unzipp_d/eyes/train/healthy'
entropion_train_dir = '/content/unzipp_d/eyes/train/entropion'
mas_train_dir = '/content/unzipp_d/eyes/train/mas'


test_dir = '/content/unzipp_d/eyes/test'
cat_test_dir = '/content/unzipp_d/eyes/test/cataract'
healthy_test_dir = '/content/unzipp_d/eyes/test/healthy'
entropion_test_dir = '/content/unzipp_d/eyes/test/entropion'
mas_test_dir = '/content/unzipp_d/eyes/test/mas'


val_dir = '/content/unzipp_d/eyes/valid'
cat_val_dir = '/content/unzipp_d/eyes/valid/cataract'
healthy_val_dir = '/content/unzipp_d/eyes/valid/healthy'
entropion_val_dir = '/content/unzipp_d/eyes/valid/entropion'
mas_val_dir = '/content/unzipp_d/eyes/valid/mas'


In [5]:
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))
num_entropion_train = len(os.listdir(entropion_train_dir))
num_mas_train = len(os.listdir(mas_train_dir))


num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))
num_entropion_test = len(os.listdir(entropion_test_dir))
num_mas_test = len(os.listdir(mas_test_dir))

num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))
num_entropion_val = len(os.listdir(entropion_val_dir))
num_mas_val = len(os.listdir(mas_val_dir))

In [6]:
print("Total Training cat Images",num_cat_train)
print("Total Training healthy Images",num_healthy_train)
print("Total Training entropion Images",num_entropion_train)
print("Total Training mas Images",num_mas_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)
print("Total test entropion Images",num_entropion_test)
print("Total test mas Images",num_mas_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)
print("Total validation entropion Images",num_entropion_val)
print("Total validation mas Images",num_mas_val)
print("--")


total_train = num_cat_train+num_healthy_train+num_entropion_train+num_mas_train
total_validation = num_cat_val+num_healthy_val+num_entropion_val+num_mas_val
total_test = num_cat_test+num_healthy_test+num_entropion_test+num_mas_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training cat Images 5290
Total Training healthy Images 1530
Total Training entropion Images 1470
Total Training mas Images 1470
--
Total test cat Images 179
Total test healthy Images 70
Total test entropion Images 70
Total test mas Images 70
--
Total validation cat Images 421
Total validation healthy Images 121
Total validation entropion Images 140
Total validation mas Images 41
--
Total Training Images 9760
--
Total Validation Images 723
--
Total Testing Images 389


In [7]:
IMG_SHAPE  = 224
batch_size = 32

In [8]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=val_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 9760 images belonging to 4 classes.
Found 722 images belonging to 4 classes.
Found 389 images belonging to 4 classes.


In [9]:
os.listdir(train_dir)

['healthy', 'cataract', 'mas', 'entropion']

listing the classes

In [10]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58889256/58889256 [==============================] - 0s 0us/step


In [11]:
for layer in pre_trained_model.layers:
  print(layer.name)
layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [12]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)

In [13]:
model = tf.keras.Model(pre_trained_model.input, x)

In [17]:
# model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])
#model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,
validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

To begin with  loss of categorical.crossentropy gave ValueError: Shapes (None, 1) and (None, 3) are incompatible  
  

Then when changed from loss of categorical.crossentropy to sparse..... recieved an error of Received a label value of 1 which is outside the valid range of [0, 1) - Python, Keras   
   
When changed to binary crossentropy recieved an error of Tensorflow estimator ValueError: `logits` and `labels` must have the same shape, received ((None, 3) vs (None, 1))